In [ ]:
import os
import pandas as pd

# List of target products and their corresponding commodity IDs
target_products = {
    "Boro-Fine": 6,               # Assuming "Boro-Fine" is rice, mapping it to id 6
    "Cow Milk (liquid)": 3,
    "Egg Farm-Red": 2,          # ID not provided in the table, will skip it
    "Green Chili (Local)": 4,
    "Potato (Local) - Old": 5,
    "Soybean Oil(loose)": 1
}

# Initialize a list to collect the data
filtered_data = []

# Load the main CSV file
df = pd.read_csv('./merged_data/final_merged_data.csv')

# Check if required columns exist
if {'Product Name and Description', 'Wholesale Price (Tk) Lowest Price - Highest Price', 'Price Date', 'Unit Wholesale'}.issubset(df.columns):
    # Filter rows based on the target products
    filtered_df = df[df['Product Name and Description'].isin(target_products.keys())]
    
    # Process each filtered row
    for _, row in filtered_df.iterrows():
        # Split the wholesale price into min and max
        wholesale_price = str(row['Wholesale Price (Tk) Lowest Price - Highest Price']).split("--")
        if len(wholesale_price) == 2:
            min_wholesale_price = float(wholesale_price[0].strip())
            max_wholesale_price = float(wholesale_price[1].strip())
        else:
            continue  # Skip if price range is not in the expected format
        
        # Get the product name and map to commodity ID
        product_name = row['Product Name and Description']
        commodity_id = target_products.get(product_name)
        
        # Only add rows with a valid commodity_id
        if commodity_id is not None:
            # Format the date to 'yyyy-MM-dd'
            formatted_date = pd.to_datetime(row['Price Date']).strftime('%Y-%m-%d')
            
            # Collect the necessary information, including commodity ID
            filtered_data.append({
                'commodity_id': commodity_id,
                'date': formatted_date,
                'min_wholesale_price': min_wholesale_price,
                'max_wholesale_price': max_wholesale_price,
                'unit': row['Unit Wholesale']
            })

# Convert the list to a DataFrame
result_df = pd.DataFrame(filtered_data)

# Drop duplicate rows based on 'commodity_id' and 'date'
result_df = result_df.drop_duplicates(subset=['commodity_id', 'date'])

# Save the DataFrame to a new CSV file
result_df.to_csv('filtered_commodity_prices.csv', index=False)

print("CSV file created successfully as 'filtered_commodity_prices.csv' with duplicates removed.")


C:\Users\user\AppData\Local\Temp\ipykernel_3360\1243817079.py:42: UserWarning: Parsing dates in  %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  formatted_date = pd.to_datetime(row['Price Date']).strftime('%Y-%m-%d')


CSV file created successfully as 'filtered_commodity_prices.csv' with duplicates removed.


In [ ]:
pip install mysql-connector-python


In [12]:
import pandas as pd
import mysql.connector

# Load the CSV data into a pandas DataFrame
data = pd.read_csv('./filtered_commodity_prices.csv')

# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='localhost',       # e.g., 'localhost'
    user='root',            # your MySQL username
    password='Lig8h0us3@',  # your MySQL password
    database='price_flow'   # name of your database
)

# Create a cursor object
cursor = conn.cursor()

# Define the SQL insert statement
insert_query = """
    INSERT INTO prices (commodity_id, date, min_price, max_price, unit)
    VALUES (%s, %s, %s, %s, %s)
"""

# Insert only the first 5 rows for testing
for index, row in data.head(5).iterrows():  # Use head(5) to limit to 5 rows
    insert_query = """
    INSERT INTO prices (commodity_id, date, min_price, max_price, unit)
    VALUES (%s, %s, %s, %s, %s)
    """
    cursor.executemany(insert_query, data[['commodity_id', 'date', 'min_wholesale_price', 'max_wholesale_price', 'unit']].values.tolist())

# Commit the transaction
conn.commit()

# Close the connection
cursor.close()
conn.close()

print("Test data inserted successfully.")


DatabaseError: 1205 (HY000): Lock wait timeout exceeded; try restarting transaction

In [13]:
import pandas as pd
import mysql.connector

# Load the CSV data into a pandas DataFrame
data = pd.read_csv('./filtered_commodity_prices.csv')

# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Lig8h0us3@',
    database='price_flow'
)

# Create a cursor object
cursor = conn.cursor()

# Define the SQL insert statement
insert_query = """
    INSERT INTO prices (commodity_id, date, min_price, max_price, unit)
    VALUES (%s, %s, %s, %s, %s)
"""

# Insert each row from the DataFrame into the table
for index, row in data.iterrows():
    cursor.execute(insert_query, (
        row['commodity_id'],
        row['date'].strip(),
        row['min_wholesale_price'],
        row['max_wholesale_price'],
        row['unit']
    ))

# Commit the transaction
conn.commit()

# Close the connection
cursor.close()
conn.close()

print("Data inserted successfully.")


DatabaseError: 1205 (HY000): Lock wait timeout exceeded; try restarting transaction